In [1]:
import os
import textwrap
from pathlib import Path
from typing import cast

import langextract as lx
import polars as pl
from langextract.data import AnnotatedDocument

In [2]:
# https://www.kaggle.com/datasets/farhan999/tokopedia-product-reviews
dataset_path = Path("datasets/tokopedia-product-reviews-2019.csv")

df = pl.read_csv(dataset_path, ignore_errors=True)
df.head()

,text,rating,category,product_name,product_id,sold,shop_id,product_url
i64,str,i64,str,str,i64,i64,i64,str
1,"""Barang sesuai pesanan dan cepa…",5,"""pertukangan""","""Staples Dekorasi Staples Kayu …",418660637,1,1740837,"""https://www.tokopedia.com/shak…"
2,"""Barang bagus harga murah""",5,"""pertukangan""","""STAPLE GUN ATS 3 WAY TACKER - …",416032545,11,1477109,"""https://www.tokopedia.com/jura…"
3,"""Paket rapi...mantap....cepat..…",5,"""pertukangan""","""STAPLE GUN ATS 3 WAY TACKER - …",416032545,11,1477109,"""https://www.tokopedia.com/jura…"
4,"""ya saya puas dgn barangnya""",5,"""pertukangan""","""ALAT STAPLES TEMBAK &#40;AIR N…",102279869,5,771395,"""https://www.tokopedia.com/kama…"
5,"""Responya luar biasa b mantap""",5,"""pertukangan""","""Isi Refill Staples Jok Kulit M…",190679689,787,969999,"""https://www.tokopedia.com/mitr…"


In [3]:
for sample in df.sample(5, seed=123).iter_rows(named=True):
    print(sample["text"])

Fast respom dan bagus barangnya. Cuman sedikit longgar dan ga dapet kabel powernya
Packing barang ok, harga murah, dapet tas yg muat 2 raket, senar dipasangin, tarikan ok keras, dipasangin grip juga, barang sampainya cepat, fisik raketnya juga bagus, mantab dipakainya, pokoknya top banget, terima kasih
semoga batre nya kuat melebihi masa garansi
berfungsi dengan baik terima kasih fast response
Barang bagus pengiriman cepet thankz


In [4]:
prompt = textwrap.dedent("""
Ekstrak review dengan deskripsi yang detail untuk 'pelayanan', 'kualitas', dan 'bonus' menggunakan attribute untuk menunjukan sentimen dan intensitasnya:

1. Ekstrak entitas sesuai dengan urutan kemunculannya pada teks
2. Setiap entitas harus memiliki atribut 'sentimen' dan 'intensitas' yang sesuai
3. Sentimen terdiri dari 'positif' atau 'negatif'. Intensitas terdiri dari 'lemah', 'sedang', dan 'kuat'.
""")

examples = [
    lx.data.ExampleData(
        text="Fast respom dan bagus barangnya. Cuman sedikit longgar dan ga dapet kabel powernya",
        extractions=[
            lx.data.Extraction(
                extraction_class="pelayanan",
                extraction_text="Fast respom",
                attributes={
                    "sentimen": "positif",
                    "intensitas": "sedang",
                },
            ),
            lx.data.Extraction(
                extraction_class="kualitas",
                extraction_text="bagus barangnya",
                attributes={
                    "sentimen": "positif",
                    "intensitas": "sedang",
                },
            ),
            lx.data.Extraction(
                extraction_class="kualitas",
                extraction_text="sedikit longgar",
                attributes={
                    "sentimen": "negatif",
                    "intensitas": "lemah",
                },
            ),
            lx.data.Extraction(
                extraction_class="bonus",
                extraction_text="ga dapet kabel powernya",
                attributes={
                    "sentimen": "negatif",
                    "intensitas": "kuat",
                },
            ),
        ],
    ),
    # lx.data.ExampleData(
    #     text="Packing barang ok, harga murah, dapet tas yg muat 2 raket, senar dipasangin, tarikan ok keras, dipasangin grip juga, barang sampainya cepat, fisik raketnya juga bagus, mantab dipakainya, pokoknya top banget, terima kasih",
    #     extractions=[
    #         lx.data.Extraction(
    #             extraction_class="pelayanan",
    #             extraction_text="Packing barang ok",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "sedang",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="harga",
    #             extraction_text="harga murah",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "sedang",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="bonus",
    #             extraction_text="dapet tas",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "tinggi",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="bonus",
    #             extraction_text="senar dipasangin",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "sedang",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="kualitas",
    #             extraction_text="tarikan ok keras",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "sedang",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="bonus",
    #             extraction_text="dipasangin grip",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "sedang",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="pelayanan",
    #             extraction_text="barang sampainya cepat",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "sedang",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="kualitas",
    #             extraction_text="fisik raketnya juga bagus",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "sedang",
    #             },
    #         ),
    #         lx.data.Extraction(
    #             extraction_class="kualitas",
    #             extraction_text="pokoknya top banget",
    #             attributes={
    #                 "sentimen": "positif",
    #                 "intensitas": "tinggi",
    #             },
    #         ),
    #     ],
    # ),
]

In [5]:
config_llamacpp = lx.factory.ModelConfig(
    model_id="hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf",
    provider="LlamaCppLanguageModel",
    provider_kwargs=dict(
        n_gpu_layers=-1,
        n_ctx=4096,
        verbose=False,
        completion_kwargs=dict(
            temperature=0,
            seed=42,
        ),
    ),
)

config_openrouter = lx.factory.ModelConfig(
    model_id="meta-llama/llama-3.3-70b-instruct:free",
    provider="OpenAILanguageModel",  # explicitly set the provider to `OpenAILanguageModel`
    provider_kwargs=dict(
        base_url="https://openrouter.ai/api/v1",
        api_key=os.environ["OPENROUTER_API_KEY"],
    ),
)

model_llamacpp = lx.factory.create_model(config_llamacpp)
model_openrouter = lx.factory.create_model(config_openrouter)

llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64 

In [6]:
group_rating = (
    df.sample(fraction=1.0, shuffle=True, seed=42)
    .with_columns(pl.col("text").str.replace_all(r"\n", " ").alias("text"))
    .group_by("rating", maintain_order=True)
    .agg(
        pl.col("text").first(),
    )
    .sort("rating")
)
group_rating.head()

rating,text
i64,str
1,"""Cacat produk nya gak bisa nyal…"
2,"""Barang baru dipakai beberapa h…"
3,"""Harga cukup murah di flash sal…"
4,"""Barang sudah sampai di rumah d…"
5,"""Barang ori sesuai pesanan, ter…"


In [7]:
for text in group_rating["text"]:
    print(text)

Cacat produk nya gak bisa nyala padahal baru beli
Barang baru dipakai beberapa hari tidak bisa di clear bekas coretannya.
Harga cukup murah di flash sale, kulit sintetis, sy kira semua produk brodo genuine leather
Barang sudah sampai di rumah dengan selamat. Dari matras dan tas matrasnya juga sama persis seperti di gambar yg diiklankan. Overall puas belanja di lapak ini. Terima kasih yaaa~
Barang ori sesuai pesanan, terima kasih


In [8]:
# pass by text

results = [
    lx.extract(
        model=model_llamacpp,
        text_or_documents=text,
        prompt_description=prompt,
        examples=examples,
        extraction_passes=2,  # multi-extraction pass; better result
        fence_output=False,
        use_schema_constraints=False,
        debug=False,
        max_workers=5,
    )
    for text in group_rating["text"]
]

result_list = cast(list[AnnotatedDocument], results)

LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=49 chars, processed=0 chars:  [00:16]
LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=71 chars, processed=0 chars:  [00:07]
LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=91 chars, processed=0 chars:  [00:16]
LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=178 chars, processed=0 chars:  [00:22]
LangExtract: model=hf:MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF:*Q4_K_M.gguf, current=39 chars, processed=0 chars:  [00:12]


In [9]:
for extracted in result_list[2].extractions:  # ty:ignore[not-iterable]
    print(f"{extracted.extraction_text=}")
    print(f"{extracted.extraction_class=}")
    print(f"{extracted.attributes=}")
    print(f"{extracted.char_interval=}")
    print()

extracted.extraction_text='Harga cukup murah di flash sale'
extracted.extraction_class='pelayanan'
extracted.attributes={'sentimen': 'positif', 'intensitas': 'sedang'}
extracted.char_interval=CharInterval(start_pos=0, end_pos=31)

extracted.extraction_text='kulit sintetis'
extracted.extraction_class='kualitas'
extracted.attributes={'sentimen': 'negatif', 'intensitas': 'sedang'}
extracted.char_interval=CharInterval(start_pos=33, end_pos=47)

extracted.extraction_text='sy kira semua produk brodo genuine leather'
extracted.extraction_class='bonus'
extracted.attributes={'sentimen': 'positif', 'intensitas': 'lemah'}
extracted.char_interval=CharInterval(start_pos=49, end_pos=91)



In [10]:
lx.io.save_annotated_documents(result_list, output_name="extractions_reviews.jsonl", output_dir=".")

for idx, result in enumerate(result_list):
    html_content = lx.visualize(result)
    with open(f"extraction_reviews_{idx}.html", "w") as f:
        if hasattr(html_content, "data"):
            f.write(html_content.data)  # For Jupyter/Colab
        else:
            f.write(html_content)

LangExtract: Saving to extractions_reviews.jsonl: 5 docs [00:00, 1396.24 docs/s]

✓ Saved 5 documents to extractions_reviews.jsonl
